<br />

# 1-D Heat and Moisture Transfer Calculation  

<br />


## 1. Overview
本プログラムは前進型有限差分法を用いた多孔質材料内の熱・水分の移動計算を行うものである。  
空気や塩の移動や結氷といった現象への拡張も可能であるが、ここでは一般的な熱と水分（水蒸気・液水）の移動計算までの使用方法について述べる。本プログラムでは以下の計算が行える。  

### 本プログラムでできること

#### 計算内容（収支式）  
- 熱計算  
- 熱、水分計算（蒸気拡散支配領域）  
- 熱・水分計算（液水領域を含む）  

#### 流量計算方法  
- 熱（固体熱伝導）（水分の相変化熱）  
- 水蒸気（圧力基準　or 水分化学ポテンシャル基準）（絶対湿度基準も実装予定）  
- 液水（圧力基準　or 水分化学ポテンシャル基準）  

#### 境界条件  
- 第一種境界条件（温湿度固定）  
- 第二種境界条件（流量固定）  
- 第三種境界条件（セル間の流量計算）  


## 2. Dependency  
本プログラムの使用するにあたり必要な作業環境およびモジュールについて説明する。

### 2.1 使用言語
- python ver.3.6

### 2.2 作業環境  
- jupyternotebook  
    ※nbimporterの導入が必須


### 2.3 必要なライブラリ  
- numpy
- matplotlib  


### 2.4 各モジュール（ファイル）の説明  

各モジュールの内容、および必要なファイル・ライブラリは以下に示すとおりである。

#### 各モジュールを統括するファイル
- 1D_heat_moisture_main.ipynb  
    内容；各モジュールを用いることでセルのセッティング、流量計算、収支計算までの一連の流れを示す。  
    必要ファイル；以下全てのモジュール  
  
  
#### Cellを構成するモジュール群
- gas_water.ipynb  
    内容；気体（水蒸気）の状態量・物質量を保持するモジュール  
    必要ファイル；moisture_conversion.ipynb  


- liquid_water.ipynb  
    内容；液体（純水）の状態量・物理量を保持するモジュール  
    必要ファイル；property_conversion.ipynb  


- solid_material.ipynb  
    内容；固体（材料骨格部）の状態量・物理量を保持するモジュール  
    必要ファイル；property_conversion.ipynb  


#### 流量計算に関するモジュール
- flux.ipynb  
    内容；熱・水分の流量計算式を管理するモジュール  
    必要ファイル；なし
   

#### 収支計算に関するモジュール  
- balance_equation.ipynb  
    内容；熱・水分の収支計算式を管理するモジュール  
    必要ファイル；なし


#### 材料の物性値に関するモジュール群   
- property_conversion.ipynb  
    内容；各物性値ファイルから必要な情報を取得し、流量計算に適した形に変換するモジュール  
    必要ファイル；vapour_pressure, 各種物性値ファイル


- van_genuchten.ipynb  
    内容；mualeam - vanGenuchtenの水分特性曲線から不飽和透水係数を算出するモデル  
    必要ファイル；なし  


#### 水分の状態量・物質量の変換に関するモジュール群  
⇒gas_waterモジュールに加えても良い気がする

- vapour_pressure.ipynb  
    内容；温度と水分化学ポテンシャルから水蒸気圧・飽和水蒸気圧・水蒸気圧の温度微分を求めるモジュール  
    必要ファイル；なし
    

- moisture_conversion.ipynb
    内容；水蒸気の水分状態（相対湿度・絶対湿度・水蒸気圧・水分化学ポテンシャル）を変換するモジュール  
    必要ファイル；vapour_pressure.ipynb 



## 3. Description

プログラムの基本的な考え方を以下に示す。  
計算では壁体を局所平衡が成り立つよう分割する。このとき分割された壁体の1つをセルと呼称する。  
セルは固体・液体・気体の3相から成り立つ。  
材料の物性値はセルごとに決定される。  
セル間の流量は互いのセルの状態量および物性値を用いることで計算される。  


<img src="picture/readme_no1.jpg" width= 60%>  

### 3.1 セルとは  

- セルとは複数の要素によって構成され、その要素数に制約はない。
- 主な構成要素は固体（材料骨格部・氷・塩結晶）、液体（水・溶液）、気体（空気・水蒸気）である。
- それぞれの構成要素は異なる物理量・状態量を保持している。
- ただし一般にセル内では局所平衡が成り立ち、3相の温度や水分の状態は平衡しているものと仮定する。 
- 材料の物性値および各種移動係数は
水分伝導率などの各種移動係数は、固体と液体、気体全ての物理量に影響を受けるため、各要素が保持するというよりはセルが有する情報とする。

### 3.2 流量計算  
- 流れとは、隣り合う2つのセルが熱力学的非平衡状態になった際に生じるものである。  
- 基本的に熱や物質の流れは隣接するセルのうち、同じ要素の間で生じるものである。（隣接するセル間では液水と材料の間で熱伝導は生じない）
- 


- セルの質点の置き方について
    セルの分割方法、質点の分割の仕方についてはいくつか方法が考えられる。
    - 質点を全てセルの中心に置く方法（収支計算が簡単）　⇔　
    - 質点を全てセルの端に置く方法（流量計算が簡単）　⇔　異なる材料の水分収支を解く際に平衡含水率をどうするのかが難しい
    - 境界面のセルのみ質点を端に置き、他はすべてセルの中央に置く方法（質点間の距離計算が複雑）　⇒　計算結果はきれいだがプログラムが膨大に
    - 質点を全てセルの中心に置き、境界面のセルのみ体積を半分にする方法（体積の計算が複雑）

### 3.3 境界条件  
壁の境界（端のセル）では境界内部で成り立つ方程式だけでは解を決定できないため、補助的な拘束条件を課す必要がある。  
境界条件は主に三種類存在し以下のように分類できる。

- 第一種境界条件（境界温湿度固定）  
- 第二種境界条件（流量固定）  
- 第三種境界条件（境界に隣接するからの流量計算）   

第三種境界条件を用いて空気と壁表面の流量計算を行う場合、セルクラスと同様に空気もクラス化することで流量の計算を行うことが好ましい。
この際セルクラスと同様、空気のクラスには空気の物性値および対流伝達率（または総合伝達率など）を保持する。

### 3.4 収支計算

### 3.5 エラーチェック

### 3.6 計算のフローチャート   
本プログラムにおける基本的な計算手順は以下のようになる。  

<img src="caluculationMethod.jpg" width= 60%>  


## 4. Demo

## 5. License

## 6. Authors

## 7. References